In [ ]:
from IPython.utils import io
with io.capture_output() as captured:  
   %run ../../2-FeatureExtraction/ContextualEmbeddings/ContextualEmbeddings.ipynb import train_contextual_embeddings, valid_contextual_embeddings, train_labels_2, valid_labels_2, train_labels_1, valid_labels_1

In [ ]:
from torch import nn
from torch.optim import Adam
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import re
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu"); print(device)

In [ ]:
train_data = TensorDataset(torch.from_numpy(train_contextual_embeddings), torch.from_numpy(train_labels_2))
valid_data = TensorDataset(torch.from_numpy(valid_contextual_embeddings), torch.from_numpy(valid_labels_2))

batch_size = 128

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=1024)

In [ ]:
class ContextualClassifier(nn.Module):

  def __init__(self, layer_1=768, layer_2=512, output=10):
    super(ContextualClassifier, self).__init__()
    self.linear_1 = nn.Linear(layer_1, layer_2)
    self.linear_2 = nn.Linear(layer_2, output)
    self.relu = nn.ReLU()

  def forward(self, contextualized_embeddings):
    linear_1_output = self.linear_1(contextualized_embeddings)
    linear_2_output = self.linear_2(self.relu(linear_1_output))
    output_layer = self.relu(linear_2_output)
    return output_layer


In [ ]:
model = ContextualClassifier().to(device)

In [ ]:
EPOCHS = 300
LR = 3e-6
N1 = 768
N2 = 512


criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)
criterion.to(device)

for epoch_num in tqdm(range(EPOCHS)):
  for train_input, train_label in train_loader:
    model.zero_grad()

    train_input = train_input.to(device)
    train_label = train_label.to(device)
    output = model(train_input)
    output = output.squeeze(1)
    batch_loss = criterion(output, train_label)

    batch_loss.backward()
    optimizer.step()

In [ ]:
output_labels = []

with torch.no_grad():
  for valid_input, valid_label in tqdm(valid_loader):
    valid_input = valid_input.to(device)
    valid_label = valid_label.to(device)
    output = model(valid_input).cpu()
    output = output.squeeze(1)
    output_labels.extend(output.argmax(dim=1).tolist())

print()
F1 = f1_score(valid_labels, output_labels, average='macro')
Report = classification_report(valid_labels, output_labels, zero_division=0)
print(Report)